# Rasterisasi Rata-rata LST (2013–2022)
Notebook ini menghitung rata-rata Land Surface Temperature (LST) selama periode 2013–2022, menginterpolasinya menjadi raster, dan menyimpannya dalam format GeoTIFF (.tif).

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_bounds

In [ ]:
# Asumsikan dataframe df sudah dimuat sebelumnya
df_avg_LST_all = df.groupby(['lon', 'lat'])['LST'].mean().reset_index()

In [ ]:
def save_overall_avg_to_tif(df, variable, filename='LST_2013_2022_Mean.tif', n_pixels=200):
    lon = df['lon'].values
    lat = df['lat'].values
    values = df[variable].values
    points = np.column_stack((lon, lat))

    # Bounding box
    min_lon, max_lon = lon.min(), lon.max()
    min_lat, max_lat = lat.min(), lat.max()

    # Buat grid
    grid_x, grid_y = np.mgrid[
        min_lon:max_lon:complex(n_pixels),
        min_lat:max_lat:complex(n_pixels)
    ]

    # Interpolasi nilai
    grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')

    # Transformasi raster
    transform = from_bounds(min_lon, min_lat, max_lon, max_lat, n_pixels, n_pixels)

    # Simpan ke GeoTIFF
    with rasterio.open(
        filename, 'w',
        driver='GTiff',
        height=grid_z.shape[1],
        width=grid_z.shape[0],
        count=1,
        dtype='float32',
        crs='EPSG:4326',
        transform=transform,
    ) as dst:
        dst.write(grid_z.T.astype('float32'), 1)

In [ ]:
# Simpan raster rata-rata LST seluruh periode
save_overall_avg_to_tif(df_avg_LST_all, variable='LST', filename='LST_2013_2022_Mean.tif')